In [2]:
# import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the paths to the train and test folders on your desktop
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# define the model architecture
model = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
# MODEL.summary

history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test accuracy:", test_acc)


Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/10
266/266 [==============================] - 53s 187ms/step - loss: 1.0530 - accuracy: 0.6555 - val_loss: 0.2596 - val_accuracy: 0.9113
Epoch 2/10
266/266 [==============================] - 37s 139ms/step - loss: 0.4831 - accuracy: 0.8494 - val_loss: 0.1673 - val_accuracy: 0.9453
Epoch 3/10
266/266 [==============================] - 35s 132ms/step - loss: 0.3625 - accuracy: 0.8878 - val_loss: 0.1213 - val_accuracy: 0.9580
Epoch 4/10
266/266 [==============================] - 35s 133ms/step - loss: 0.2920 - accuracy: 0.9089 - val_loss: 0.0996 - val_accuracy: 0.9680
Epoch 5/10
266/266 [==============================] - 35s 132ms/step - loss: 0.2561 - accuracy: 0.9201 - val_loss: 0.0874 - val_accuracy: 0.9720
Epoch 6/10
266/266 [==============================] - 34s 127ms/step - loss: 0.2272 - accuracy: 0.9296 - val_loss: 0.0630 - val_accuracy: 0.9803
Epoch 7/10
266/266 [=======================

In [3]:
# extract features from the layer before the output of the CNN for both train and test sets
train_features = model.predict(train_generator)
test_features = model.predict(test_generator)

# reshape the feature maps from 4D to 2D for classification
train_features = np.reshape(train_features, (train_generator.samples, -1))
test_features = np.reshape(test_features, (test_generator.samples, -1))

# use the extracted feature vectors for classification using k-NN, decision tree, and random forest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = {
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

for name, clf in classifiers.items():
    clf.fit(train_features, train_generator.labels)
    score = clf.score(test_features, test_generator.labels)
    print("{} score: {:.2f}".format(name, score))

# use the extracted feature vectors for regression using linear regression, gradient boosted trees, and XGBoost
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

regressors = {
    "Linear Regression": LinearRegression(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor()
}

for name, reg in regressors.items():
    reg.fit(train_features, train_generator.labels)
    score = reg.score(test_features, test_generator.labels)
    print("{} score: {:.2f}".format(name, score))


47/47 [==============================] - 4s 78ms/step


C:\Users\chand\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


k-NN score: 0.11
Decision Tree score: 0.10
Random Forest score: 0.12
Linear Regression score: -0.00
Gradient Boosting score: -0.01
XGBoost score: -0.14


In [5]:
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Dropout

# define the model architecture with regularization and dropout layers
model = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu", kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation="relu", kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.Dropout(0.2),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# define early stopping callback
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# train the model with data augmentation and early stopping
history = model.fit(train_generator, epochs=15, validation_data=test_generator, callbacks=[early_stop])


Epoch 1/15
266/266 [==============================] - 45s 159ms/step - loss: 7.8434 - accuracy: 0.0994 - val_loss: 2.4375 - val_accuracy: 0.1000
Epoch 2/15
266/266 [==============================] - 40s 149ms/step - loss: 2.4371 - accuracy: 0.0984 - val_loss: 2.4359 - val_accuracy: 0.1000
Epoch 3/15
266/266 [==============================] - 41s 153ms/step - loss: 2.4350 - accuracy: 0.0981 - val_loss: 2.4345 - val_accuracy: 0.1000
Epoch 4/15
266/266 [==============================] - 41s 155ms/step - loss: 2.4335 - accuracy: 0.0965 - val_loss: 2.4325 - val_accuracy: 0.1000
Epoch 5/15
266/266 [==============================] - 45s 168ms/step - loss: 2.4339 - accuracy: 0.0954 - val_loss: 2.4342 - val_accuracy: 0.1000
Epoch 6/15
266/266 [==============================] - 39s 147ms/step - loss: 2.4343 - accuracy: 0.0982 - val_loss: 2.4345 - val_accuracy: 0.1000
Epoch 7/15
266/266 [==============================] - 39s 148ms/step - loss: 2.4343 - accuracy: 0.0951 - val_loss: 2.4343 - val_ac